<a href="https://colab.research.google.com/github/zihan07/Research-of-Semantic-similarity-between-Quran-verses-with-Biographies/blob/Final-Code-of-the-Research/Threshold_Accuracy_with_mecca_and_medina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **11**

In [104]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v1 b1.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v1 b1.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b1.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.170000
1  2nd Quartile (Q2, Median)        53.940000
2          3rd Quartile (Q3)        58.520000
3                    Average        54.244605
Threshold 1: 49.17
Threshold 2: 53.94
Threshold 3: 58.52
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b1.xlsx


# **13**

In [105]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v1 b3.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v1 b3.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b3.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        47.920000
1  2nd Quartile (Q2, Median)        52.320000
2          3rd Quartile (Q3)        56.660000
3                    Average        52.549344
Threshold 1: 47.92
Threshold 2: 52.32
Threshold 3: 56.66
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b3.xlsx


# **15**

In [106]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v1 b5.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v1 b5.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b5.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        46.940000
1  2nd Quartile (Q2, Median)        51.020000
2          3rd Quartile (Q3)        55.140000
3                    Average        51.288604
Threshold 1: 46.94
Threshold 2: 51.02
Threshold 3: 55.14
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b5.xlsx


# **17**

In [107]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v1 b7.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v1 b7.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b7.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        46.560000
1  2nd Quartile (Q2, Median)        50.530000
2          3rd Quartile (Q3)        54.860000
3                    Average        50.972368
Threshold 1: 46.56
Threshold 2: 50.53
Threshold 3: 54.86
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v1 b7.xlsx


# **31**

In [108]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v3 b1.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v3 b1.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b1.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.800000
1  2nd Quartile (Q2, Median)        53.480000
2          3rd Quartile (Q3)        57.360000
3                    Average        53.745594
Threshold 1: 49.8
Threshold 2: 53.48
Threshold 3: 57.36
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b1.xlsx


# **33**

In [109]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v3 b3.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v3 b3.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b3.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.680000
1  2nd Quartile (Q2, Median)        53.460000
2          3rd Quartile (Q3)        57.407500
3                    Average        53.725009
Threshold 1: 49.68
Threshold 2: 53.46
Threshold 3: 57.4075
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b3.xlsx


# **35**

In [110]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v3 b5.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v3 b5.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b5.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.047500
1  2nd Quartile (Q2, Median)        52.760000
2          3rd Quartile (Q3)        56.632500
3                    Average        52.991801
Threshold 1: 49.0475
Threshold 2: 52.76
Threshold 3: 56.6325
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b5.xlsx


# **37**

In [111]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v3 b7.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v3 b7.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b7.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        48.720000
1  2nd Quartile (Q2, Median)        52.480000
2          3rd Quartile (Q3)        56.280000
3                    Average        52.719795
Threshold 1: 48.72
Threshold 2: 52.48
Threshold 3: 56.28
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v3 b7.xlsx


# **51**

In [112]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v5 b1.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v5 b1.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b1.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.720000
1  2nd Quartile (Q2, Median)        53.180000
2          3rd Quartile (Q3)        56.715000
3                    Average        53.373135
Threshold 1: 49.72
Threshold 2: 53.18
Threshold 3: 56.715
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b1.xlsx


# **53**

In [113]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v5 b3.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v5 b3.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b3.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        50.120000
1  2nd Quartile (Q2, Median)        53.690000
2          3rd Quartile (Q3)        57.270000
3                    Average        53.888682
Threshold 1: 50.12
Threshold 2: 53.69
Threshold 3: 57.27
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b3.xlsx


# **55**

In [114]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v5 b5.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v5 b5.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b5.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.570000
1  2nd Quartile (Q2, Median)        53.100000
2          3rd Quartile (Q3)        56.670000
3                    Average        53.332922
Threshold 1: 49.57
Threshold 2: 53.1
Threshold 3: 56.67
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b5.xlsx


# **57**

In [115]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v5 b7.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v5 b7.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b7.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.240000
1  2nd Quartile (Q2, Median)        52.820000
2          3rd Quartile (Q3)        56.422500
3                    Average        53.065639
Threshold 1: 49.24
Threshold 2: 52.82
Threshold 3: 56.4225
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v5 b7.xlsx


# **71**

In [116]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v7 b1.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v7 b1.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b1.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)         49.66000
1  2nd Quartile (Q2, Median)         53.05000
2          3rd Quartile (Q3)         56.58000
3                    Average         53.26116
Threshold 1: 49.66
Threshold 2: 53.05
Threshold 3: 56.58
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b1.xlsx


# **73**

In [117]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v7 b3.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v7 b3.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b3.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)         50.17000
1  2nd Quartile (Q2, Median)         53.70000
2          3rd Quartile (Q3)         57.25000
3                    Average         53.91641
Threshold 1: 50.17
Threshold 2: 53.7
Threshold 3: 57.25
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b3.xlsx


# **75**

In [118]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3

# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v7 b5.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v7 b5.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b5.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.750000
1  2nd Quartile (Q2, Median)        53.230000
2          3rd Quartile (Q3)        56.770000
3                    Average        53.477612
Threshold 1: 49.75
Threshold 2: 53.23
Threshold 3: 56.77
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b5.xlsx


# **77**

In [119]:
import pandas as pd

def evaluate_predictions(df, threshold1, threshold2, threshold3):
    # Filtering based on thresholds
    df_filtered1 = df[df['Best Match Score (as percentage)'] >= threshold1]
    df_filtered2 = df[df['Best Match Score (as percentage)'] >= threshold2]
    df_filtered3 = df[df['Best Match Score (as percentage)'] >= threshold3]

    def compute_metrics(df_filtered):
        TP = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        TN = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Mecca
        TP_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FP_mecca = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()
        FN_mecca = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()

        # Confusion matrix for Medina
        TP_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FP_medina = ((df_filtered['Actual Period'] == 'Mecca') & (df_filtered['Predicted Period'] == 'Medina')).sum()
        FN_medina = ((df_filtered['Actual Period'] == 'Medina') & (df_filtered['Predicted Period'] == 'Mecca')).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
       # Precision, Recall, and F1 Score for Mecca
        precision_mecca = TP_mecca / (TP_mecca + FP_mecca) if (TP_mecca + FP_mecca) > 0 else 0
        recall_mecca = TP_mecca / (TP_mecca + FN_mecca) if (TP_mecca + FN_mecca) > 0 else 0
        f1_mecca = 2 * (precision_mecca * recall_mecca) / (precision_mecca + recall_mecca) if (precision_mecca + recall_mecca) > 0 else 0

        # Precision, Recall, and F1 Score for Medina
        precision_medina = TP_medina / (TP_medina + FP_medina) if (TP_medina + FP_medina) > 0 else 0
        recall_medina = TP_medina / (TP_medina + FN_medina) if (TP_medina + FN_medina) > 0 else 0
        f1_medina = 2 * (precision_medina * recall_medina) / (precision_medina + recall_medina) if (precision_medina + recall_medina) > 0 else 0

        return {
            'Accuracy': accuracy,
            'Precision (Mecca)': precision_mecca,
            'Recall (Mecca)': recall_mecca,
            'F1 Score (Mecca)': f1_mecca,
            'Precision (Medina)': precision_medina,
            'Recall (Medina)': recall_medina,
            'F1 Score (Medina)': f1_medina
        }

    metrics1 = compute_metrics(df_filtered1)
    metrics2 = compute_metrics(df_filtered2)
    metrics3 = compute_metrics(df_filtered3)

    return metrics1, metrics2, metrics3
# Load dataset
file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/v7 b7.xlsx'
df = pd.read_excel(file_path)

# Load thresholds from the Excel sheet
thresholds_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/C_v7 b7.xlsx'  # Replace with your thresholds file path
thresholds_df = pd.read_excel(thresholds_file_path)

# Debugging: Print the thresholds DataFrame to verify its structure
print("Thresholds DataFrame:")
print(thresholds_df)

# Extract thresholds from the second column (assuming the first column is an index or header)
threshold1 = thresholds_df.iloc[0, 1]  # First row, second column
threshold2 = thresholds_df.iloc[1, 1]  # Second row, second column
threshold3 = thresholds_df.iloc[2, 1]  # Third row, second column

# Debugging: Print the thresholds to verify
print(f"Threshold 1: {threshold1}")
print(f"Threshold 2: {threshold2}")
print(f"Threshold 3: {threshold3}")

# Evaluate with the thresholds from the Excel sheet
metrics1, metrics2, metrics3 = evaluate_predictions(df, threshold1, threshold2, threshold3)

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Threshold': [threshold1, threshold2, threshold3],
    'Accuracy': [metrics1['Accuracy'], metrics2['Accuracy'], metrics3['Accuracy']],
    'Precision (Mecca)': [metrics1['Precision (Mecca)'], metrics2['Precision (Mecca)'], metrics3['Precision (Mecca)']],
    'Recall (Mecca)': [metrics1['Recall (Mecca)'], metrics2['Recall (Mecca)'], metrics3['Recall (Mecca)']],
    'F1 Score (Mecca)': [metrics1['F1 Score (Mecca)'], metrics2['F1 Score (Mecca)'], metrics3['F1 Score (Mecca)']],
    'Precision (Medina)': [metrics1['Precision (Medina)'], metrics2['Precision (Medina)'], metrics3['Precision (Medina)']],
    'Recall (Medina)': [metrics1['Recall (Medina)'], metrics2['Recall (Medina)'], metrics3['Recall (Medina)']],
    'F1 Score (Medina)': [metrics1['F1 Score (Medina)'], metrics2['F1 Score (Medina)'], metrics3['F1 Score (Medina)']]
})

# Save the DataFrame to an Excel file
output_file_path = '/content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b7.xlsx'
metrics_df.to_excel(output_file_path, index=False)

print(f"Metrics saved to {output_file_path}")

Thresholds DataFrame:
                    Quartile  Threshold Value
0          1st Quartile (Q1)        49.420000
1  2nd Quartile (Q2, Median)        52.915000
2          3rd Quartile (Q3)        56.560000
3                    Average        53.209911
Threshold 1: 49.42
Threshold 2: 52.915
Threshold 3: 56.56
Metrics saved to /content/drive/MyDrive/Quran-vs-biography-semantic-similarity-CSE498/U_all-MiniLM-L12-v2/O_v7 b7.xlsx
